# Model Server

Model Serving provides a solution to host machine learning / deep learning (ML/DL) models as REST endpoints that are updated automatically, enabling data science teams to own the end-to-end lifecycle of a real-time machine learning model from training to production.

**`model_server.ipynb`** deploy any classifier model that has been pickled (cloudpickle).

For more demonstrations :

1. **[lightgbm-project](https://github.com/yjb-ds/lightgbm-project)**
2. **[demo-sklearn-project](https://github.com/yjb-ds/demo-sklearn-project)**
3. **[demo-xgb-project](https://github.com/yjb-ds/demo-xgb-project/tree/functions)**

# Example models locally and deploy

The sklearn-project generated one or more models that will be deployed in the server project `sklearn-servers`

## locally

In [1]:
import os
import mlrun
from model_server import ClassifierModel
from cloudpickle import load
from sklearn.datasets import load_iris

In [2]:
model = "https://s3.wasabisys.com/iguazio/models/iris/model.pkl"

iris = load_iris()

x = iris["data"].tolist()
y = iris["target"]

my_server = ClassifierModel("classifier", model_dir=model)
my_server.load()

a = my_server.predict({"instances": x})
assert len(a) == 150

## Document and save

In [3]:
import yaml

with open("item.yaml") as item_file:
    items = yaml.load(item_file, Loader=yaml.FullLoader)

In [4]:
fn = mlrun.new_model_server(
    name=items["name"],
    model_class="ClassifierModel",
    filename=items["spec"]["filename"],
)

fn.spec.description = items["description"]
fn.metadata.categories = items["categories"]
fn.metadata.labels = labels = items["labels"]
fn.spec.image = items["spec"]["image"]
fn.spec.requirements = items["spec"]["requirements"]
fn.spec.kind = items["spec"]["kind"]

fn.export("model_server.yaml")

> 2021-02-17 08:17:19,639 [info] function spec saved to path: model_server.yaml


##  Deploy server

In [5]:
user_name = os.getenv("V3IO_USER_NAME")
artifact_path = mlrun.set_environment(
    api_path="http://mlrun-api:8080", artifact_path=os.path.abspath("./")
)
fn.apply(mlrun.mount_v3io())
fn.set_envs(
    {
        "SERVING_MODEL_iris_dataset_v1": model,
        "INFERENCE_STREAM": "users/{}/tststream".format(user_name),
    }
)

address = fn.deploy(project="sk-project")

> 2021-02-17 08:17:19,666 [info] Starting remote function deploy
2021-02-17 08:17:19  (info) Deploying function
2021-02-17 08:17:19  (info) Building
2021-02-17 08:17:19  (info) Staging files and preparing base images
2021-02-17 08:17:19  (info) Building processor image
2021-02-17 08:17:24  (info) Build complete
2021-02-17 08:17:32  (info) Function deploy complete
> 2021-02-17 08:17:32,329 [info] function deployed, address=default-tenant.app.vmdev36.lab.iguazeng.com:32072


##  Test server

In [6]:
predict_url = address + "/iris_dataset_v1/predict"
my_data = """{"instances":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}"""
!curl {predict_url} -d '{my_data}'

[0, 2]